In [1]:
import pandas as pd
from sqlalchemy import select
from sqlalchemy.sql import join
from models import Player, TradPlayerStats, AdvPlayerStats, TradTeamStats, AdvTeamStats, Game
from data_manager import DataManager
import date_utils as dt_mng


In [2]:
class Player_Data:
    def __init__(self, player_name, season, start_at):
        self.dm = DataManager()
        self.engine = self.dm.get_database_engine()
        session = self.dm.get_session()
        self.player =  session.query(Player).filter(Player.name == player_name).one_or_none()
        self.player_name = player_name
        self.start_at = start_at
        if not self.player:
            raise KeyError(f"No player found with the name {player_name}.")
        
        self.season = season[:5] + season[-2:] if len(season) == 9 else season
        self.team_games = self.get_team_games()
        self.stats = self.get_individual_game_stats()
        session.close()

    def get_team_games(self):
        session = self.dm.get_session()
        try:
            if self.start_at == "Regular Season" or self.start_at == "Playoffs":
                query = (
                    select(
                        TradTeamStats,
                        AdvTeamStats,
                        Game.date.label('game_date')
                        )
                        .select_from(
                            join(TradTeamStats, Game, TradTeamStats.game_id == Game.id)
                            .join(AdvTeamStats, (AdvTeamStats.game_id == Game.id) & (AdvTeamStats.team_id == TradTeamStats.team_id))
                        )
                        .where(TradTeamStats.team_id == self.player.team_id)
                        .where(AdvTeamStats.team_id == self.player.team_id)
                        .where(Game.season_type == self.start_at)
                    )
                return pd.read_sql(query, self.engine)
            
            elif self.start_at.isinstance(int):
                start_date = dt_mng.get_date_n_days_ago(self.start_at)
                query = (
                    select(
                        TradTeamStats,
                        AdvTeamStats,
                        Game.date.label('game_date')
                    )
                    .select_from(
                        join(TradTeamStats, Game, TradTeamStats.game_id == Game.id)
                        .join(AdvTeamStats, (AdvTeamStats.game_id == Game.id) & (AdvTeamStats.team_id == TradTeamStats.team_id))
                    )
                    .where(TradTeamStats.team_id == self.player.team_id)
                    .where(AdvTeamStats.team_id == self.player.team_id)
                    .where(Game.date >= start_date)
                    .where(Game.date < dt_mng.get_todays_date())
                    )
                return pd.read_sql(query, self.engine)
        finally:
            session.close()
            
    def get_player_games(self):
        session = self.dm.get_session()
        try:
            if self.start_at in ["Regular Season", "Playoffs"]:
                query = (
                    select(
                        Player,
                        AdvPlayerStats,
                        TradPlayerStats,
                        Game.date.label('game_date')
                    )
                    .select_from(
                        join(TradPlayerStats, Game, TradPlayerStats.game_id == Game.id)
                        .join(AdvPlayerStats, (AdvPlayerStats.game_id == Game.id) & (AdvPlayerStats.player_id == TradPlayerStats.player_id))
                        .join(Player, Player.id == TradPlayerStats.player_id)
                    )
                    .where(Player.id == self.player.id)
                    .where(Game.season_type == self.start_at)
                )
            elif isinstance(self.start_at, int):
                start_date = dt_mng.get_date_n_days_ago(self.start_at)
                query = (
                    select(
                        Player,
                        AdvPlayerStats,
                        TradPlayerStats,
                        Game.date.label('game_date')
                    )
                    .select_from(
                        join(TradPlayerStats, Game, TradPlayerStats.game_id == Game.id)
                        .join(AdvPlayerStats, (AdvPlayerStats.game_id == Game.id) & (AdvPlayerStats.player_id == TradPlayerStats.player_id))
                        .join(Player, Player.id == TradPlayerStats.player_id)
                    )
                    .where(Player.id == self.player.id)
                    .where(Game.date >= start_date)
                    .where(Game.date < get_todays_date())
                )

            return pd.read_sql(query, self.engine)
        finally:
            session.close()


In [10]:
pd.set_option("display.max_columns", None)
taken_parlays = pd.read_csv("parlays/taken_parlays_5_21_2024_partial.csv", index_col=0)

taken_parlays['PROPS'] = taken_parlays['PLAYER'] + "-" + taken_parlays['STAT'] + "-" + taken_parlays['TYPE']
n_parlays = len(taken_parlays['PARLAY_ID'].unique())
print(f"Parlays taken: {n_parlays}")
props = taken_parlays['PROPS'].unique()
n_props = len(props)
print(f"Props involved: {n_props}")



Parlays taken: 8
Props involved: 14


In [98]:
from excel_funcs import save_as_excel_workbook

player_name = "Tyrese Haliburton"
season = "2023-24"
games_start_last_n_or_date_or_season_type = "Playoffs"
player_data = Player_Data(
    player_name,
    season, 
    games_start_last_n_or_date_or_season_type
    )
team_stats = player_data.team_games
player_stats = player_data.stats
filename = f"data_pile/{player_data.player_name}_{player_data.start_at}_{player_data.season}"
save_as_excel_workbook([team_stats, player_stats], filename)

1630169


In [99]:

dm = DataManager()
season = "2023-24"
season_type = "Playoffs"
players = dm.get_players_from_db()
players_x = [player for player in players if player.name == "Anthony Edwards"]
test_player = players_x[0]
print(test_player.team_id)
players = [player for player in players if player.team_id == test_player.team_id]
player_names = [player.name for player in players]
print(player_names)

for player_name in player_names:
    season = "2023-24"
    games_start_last_n_or_date_or_season_type = "Playoffs"
    player_data = Player_Data(
        player_name,
        season, 
        games_start_last_n_or_date_or_season_type
        )
    team_stats = player_data.team_games
    player_stats = player_data.stats
    filename = f"data_pile/{player_data.player_name}_{player_data.start_at}_{player_data.season}"
    save_as_excel_workbook([team_stats, player_stats], filename)

1610612750
['Jaylen Clark', 'Kyle Anderson', 'Jaden McDaniels', 'Anthony Edwards', 'Jordan McLaughlin', 'Wendell Moore Jr.', 'Josh Minott', 'Nickeil Alexander-Walker', 'Mike Conley', 'Naz Reid', 'Daishen Nix', 'Monte Morris', 'T.J. Warren', 'Rudy Gobert', 'Karl-Anthony Towns', 'Leonard Miller', 'Luka Garza']
